In [1]:
from ns3gym import ns3env
from comet_ml import Experiment
import tqdm
import subprocess
from collections import deque
import numpy as np

# from agents.ddpg.agent import Agent, Config
from agents.dqn.agent import Agent
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.1) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Basic constants and setting up environment

In [2]:
simTime = 10 # seconds
stepTime = 0.1  # seconds
history_length = 10

EPISODE_COUNT = 30
steps_per_ep = int(simTime/stepTime)

SCRIPT_RUNNING = False

script_exec_command = f'../../waf --run "linear-mesh --simTime={simTime} --envStepTime={stepTime} --historyLength={history_length}"'
print("Steps per episode:", steps_per_ep)

Steps per episode: 100


In [3]:
port = 5555
seed = 0

simArgs = {"--simTime": simTime,
           "--testArg": 123,
           "--nodeNum": 5,
           "--distance": 500}
debug = False

subprocess.Popen(['bash', '-c', script_exec_command])
SCRIPT_RUNNING = True

env = ns3env.Ns3Env(port=port, stepTime=stepTime, startSim=0, simSeed=seed, simArgs=simArgs, debug=debug)

Waiting for simulation script to connect on port: tcp://localhost:5555
Please start proper ns-3 simulation script using ./waf --run "..."


In [4]:
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Observation space shape: Box(10,)
Action space shape: Box(1,)


### Creating and training agent

In [5]:
import tensorflow as tf

class Network(QNetworkTf):
    def _inference(self):
        with tf.variable_scope("inference_"+self.name):
            layer = tf.layers.dense(self.input, 1024, activation=tf.nn.relu)
            layer = tf.layers.dense(layer, 512, activation=tf.nn.relu)
#             layer = tf.layers.batch_normalization(layer)
            layer = tf.layers.dense(layer, 512, activation=tf.nn.relu)
            layer = tf.layers.dense(layer, 256, activation=tf.nn.relu)
            layer = tf.layers.dense(layer, 256, activation=tf.nn.relu)
            layer = tf.layers.dense(layer, 64, activation=tf.nn.relu)
            layer = tf.layers.dense(layer, self.action_size)
        return layer

In [6]:
if not SCRIPT_RUNNING:
    subprocess.Popen(['bash', '-c', script_exec_command])
    SCRIPT_RUNNING = True

agent = Agent(Network, history_length, 3)
agent.set_epsilon(0.9, 0.01, 25)
teacher = Teacher(agent, env, 1)
teacher.train(EPISODE_COUNT, simTime, stepTime, "Inp: window Mb sent", "Rew: normalized speed", "Warmup", "Reward - 1/2", "DQN")

SCRIPT_RUNNING = False

Action space: 3


COMET INFO: old comet version (1.0.45) detected. current: 1.0.47 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wwydmanski/rl-in-wifi/798efb35e12c4598951dae40c43d623d

 48%|█████████████████████████████▊                                | 48/100 [00:31<00:33,  1.54it/s, mb_sent=139.42 Mb]


TypeError: float() argument must be a string or a number, not 'dict'